In [1]:
# Slighlty updated for Keras 2.0 APIs
# - by Kyeong Soo (Joseph) Kim

import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
path_train = "./UJIndoorLoc/trainingData2.csv"
path_validation = "./UJIndoorLoc/validationData2.csv"

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [7]:
epochs = 20
batch_size = 10
input_size = 520
num_classes = 13
nums_hidden_nodes = [16, 32, 64, 128, 256, 512]
# nums_hidden_nodes = [16]
# dropout_rates = np.arange(11)*0.05 # 0.0, 0.05,...,0.5
dropout_rates = [0.0, 0.2, 0.5] # for test
# dropout_rates = [0.5] # for test
losses = np.zeros((len(nums_hidden_nodes), len(dropout_rates)))
accuracies = np.zeros_like(losses)

In [8]:
def classifier(num_hidden_nodes, dropout_rate):
    model = Sequential()
    model.add(Dense(num_hidden_nodes, input_dim=input_size, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(256, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(128, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(64, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax', use_bias=True))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [9]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        c = classifier(nums_hidden_nodes[nh], dropout_rates[dr])
        startTime = timer()
        c.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=batch_size, epochs=epochs)
        elapsedTime = timer() - startTime
        print("Model trained with nh=%d & dr=%.2f in %e s.\n" % (nums_hidden_nodes[nh], dropout_rates[dr], elapsedTime))
        loss, acc = c.evaluate(test_AP_features, test_labels)
        losses[nh, dr] = loss
        accuracies[nh, dr] = acc

Train on 13961 samples, validate on 5969 samples
Epoch 1/20
13961/13961 [==============================] - 2s - loss: 0.2760 - acc: 0.9288 - val_loss: 0.0536 - val_acc: 0.9881
Epoch 2/20
13961/13961 [==============================] - 2s - loss: 0.0383 - acc: 0.9916 - val_loss: 0.0411 - val_acc: 0.9866
Epoch 3/20
13961/13961 [==============================] - 2s - loss: 0.0248 - acc: 0.9940 - val_loss: 0.0372 - val_acc: 0.9866
Epoch 4/20
13961/13961 [==============================] - 2s - loss: 0.0187 - acc: 0.9943 - val_loss: 0.0333 - val_acc: 0.9873
Epoch 5/20
13961/13961 [==============================] - 2s - loss: 0.0144 - acc: 0.9953 - val_loss: 0.0316 - val_acc: 0.9899
Epoch 6/20
13961/13961 [==============================] - 2s - loss: 0.0135 - acc: 0.9953 - val_loss: 0.0219 - val_acc: 0.9918
Epoch 7/20
13961/13961 [==============================] - 2s - loss: 0.0109 - acc: 0.9956 - val_loss: 0.0246 - val_acc: 0.9913
Epoch 8/20
13961/13961 [==============================] - 2s -

13961/13961 [==============================] - 3s - loss: 0.2116 - acc: 0.9441 - val_loss: 0.0441 - val_acc: 0.9899
Epoch 2/20
13961/13961 [==============================] - 2s - loss: 0.0323 - acc: 0.9911 - val_loss: 0.0448 - val_acc: 0.9863
Epoch 3/20
13961/13961 [==============================] - 3s - loss: 0.0217 - acc: 0.9942 - val_loss: 0.0234 - val_acc: 0.9931
Epoch 4/20
13961/13961 [==============================] - 3s - loss: 0.0170 - acc: 0.9950 - val_loss: 0.0354 - val_acc: 0.9869
Epoch 5/20
13961/13961 [==============================] - 3s - loss: 0.0115 - acc: 0.9962 - val_loss: 0.0211 - val_acc: 0.9926
Epoch 6/20
13961/13961 [==============================] - 3s - loss: 0.0087 - acc: 0.9969 - val_loss: 0.0184 - val_acc: 0.9925
Epoch 7/20
13961/13961 [==============================] - 3s - loss: 0.0130 - acc: 0.9958 - val_loss: 0.0163 - val_acc: 0.9933
Epoch 8/20
13961/13961 [==============================] - 3s - loss: 0.0077 - acc: 0.9964 - val_loss: 0.0169 - val_acc: 0.

13961/13961 [==============================] - 2s - loss: 0.1637 - acc: 0.9547 - val_loss: 0.0409 - val_acc: 0.9898
Epoch 2/20
13961/13961 [==============================] - 2s - loss: 0.0291 - acc: 0.9916 - val_loss: 0.0430 - val_acc: 0.9858
Epoch 3/20
13961/13961 [==============================] - 2s - loss: 0.0189 - acc: 0.9941 - val_loss: 0.0270 - val_acc: 0.9908
Epoch 4/20
13961/13961 [==============================] - 2s - loss: 0.0199 - acc: 0.9937 - val_loss: 0.0420 - val_acc: 0.9864
Epoch 5/20
13961/13961 [==============================] - 2s - loss: 0.0150 - acc: 0.9948 - val_loss: 0.0311 - val_acc: 0.9906
Epoch 6/20
13961/13961 [==============================] - 2s - loss: 0.0088 - acc: 0.9963 - val_loss: 0.0291 - val_acc: 0.9901
Epoch 7/20
13961/13961 [==============================] - 2s - loss: 0.0087 - acc: 0.9965 - val_loss: 0.0236 - val_acc: 0.9930
Epoch 8/20
13961/13961 [==============================] - 2s - loss: 0.0131 - acc: 0.9961 - val_loss: 0.0255 - val_acc: 0.

13961/13961 [==============================] - 4s - loss: 0.1256 - acc: 0.9645 - val_loss: 0.0513 - val_acc: 0.9832
Epoch 2/20
13961/13961 [==============================] - 3s - loss: 0.0284 - acc: 0.9912 - val_loss: 0.0313 - val_acc: 0.9893
Epoch 3/20
13961/13961 [==============================] - 3s - loss: 0.0209 - acc: 0.9932 - val_loss: 0.0300 - val_acc: 0.9889
Epoch 4/20
13961/13961 [==============================] - 3s - loss: 0.0635 - acc: 0.9905 - val_loss: 0.0277 - val_acc: 0.9908
Epoch 5/20
13961/13961 [==============================] - 3s - loss: 0.0152 - acc: 0.9944 - val_loss: 0.0246 - val_acc: 0.9911
Epoch 6/20
13961/13961 [==============================] - 3s - loss: 0.0095 - acc: 0.9961 - val_loss: 0.0283 - val_acc: 0.9908
Epoch 7/20
13961/13961 [==============================] - 3s - loss: 0.0094 - acc: 0.9962 - val_loss: 0.0191 - val_acc: 0.9933
Epoch 8/20
13961/13961 [==============================] - 3s - loss: 0.0096 - acc: 0.9962 - val_loss: 0.0309 - val_acc: 0.

13961/13961 [==============================] - 5s - loss: 0.1147 - acc: 0.9673 - val_loss: 0.0418 - val_acc: 0.9859
Epoch 2/20
13961/13961 [==============================] - 5s - loss: 0.0306 - acc: 0.9910 - val_loss: 0.0292 - val_acc: 0.9898
Epoch 3/20
13961/13961 [==============================] - 7s - loss: 0.0241 - acc: 0.9927 - val_loss: 0.0278 - val_acc: 0.9901
Epoch 4/20
13961/13961 [==============================] - 7s - loss: 0.0216 - acc: 0.9930 - val_loss: 0.0449 - val_acc: 0.9873
Epoch 5/20
13961/13961 [==============================] - 5s - loss: 0.0140 - acc: 0.9951 - val_loss: 0.0217 - val_acc: 0.9923
Epoch 6/20
13961/13961 [==============================] - 5s - loss: 0.0131 - acc: 0.9959 - val_loss: 0.0171 - val_acc: 0.9936
Epoch 7/20
13961/13961 [==============================] - 5s - loss: 0.0108 - acc: 0.9966 - val_loss: 0.0677 - val_acc: 0.9807
Epoch 8/20
13961/13961 [==============================] - 5s - loss: 0.0192 - acc: 0.9936 - val_loss: 0.0190 - val_acc: 0.

13961/13961 [==============================] - 11s - loss: 0.1073 - acc: 0.9672 - val_loss: 0.0342 - val_acc: 0.9879
Epoch 2/20
13961/13961 [==============================] - 10s - loss: 0.0306 - acc: 0.9897 - val_loss: 0.0292 - val_acc: 0.9910
Epoch 3/20
13961/13961 [==============================] - 10s - loss: 0.0407 - acc: 0.9883 - val_loss: 0.0294 - val_acc: 0.9903
Epoch 4/20
13961/13961 [==============================] - 10s - loss: 0.0204 - acc: 0.9930 - val_loss: 0.0719 - val_acc: 0.9811
Epoch 5/20
13961/13961 [==============================] - 10s - loss: 0.0211 - acc: 0.9933 - val_loss: 0.0300 - val_acc: 0.9916
Epoch 6/20
13961/13961 [==============================] - 10s - loss: 0.0201 - acc: 0.9936 - val_loss: 0.0581 - val_acc: 0.9843
Epoch 7/20
13961/13961 [==============================] - 10s - loss: 0.0238 - acc: 0.9933 - val_loss: 0.0618 - val_acc: 0.9858
Epoch 8/20
13961/13961 [==============================] - 10s - loss: 0.0242 - acc: 0.9928 - val_loss: 0.0283 - val

In [10]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        print("nh=%d & dr=%.2f: loss=%e, accuracy=%e" % (nums_hidden_nodes[nh], dropout_rates[dr], losses[nh, dr], accuracies[nh, dr]))

nh=16 & dr=0.00: loss=1.283800e+00, accuracy=7.416742e-01
nh=16 & dr=0.20: loss=1.673727e+00, accuracy=6.759676e-01
nh=16 & dr=0.50: loss=1.772957e+00, accuracy=6.444644e-01
nh=32 & dr=0.00: loss=1.505690e+00, accuracy=7.335734e-01
nh=32 & dr=0.20: loss=1.572381e+00, accuracy=7.155716e-01
nh=32 & dr=0.50: loss=1.882560e+00, accuracy=6.975698e-01
nh=64 & dr=0.00: loss=1.423406e+00, accuracy=7.470747e-01
nh=64 & dr=0.20: loss=1.680462e+00, accuracy=7.182718e-01
nh=64 & dr=0.50: loss=2.209932e+00, accuracy=6.966697e-01
nh=128 & dr=0.00: loss=1.386000e+00, accuracy=8.046805e-01
nh=128 & dr=0.20: loss=1.776332e+00, accuracy=7.704770e-01
nh=128 & dr=0.50: loss=1.947697e+00, accuracy=7.407741e-01
nh=256 & dr=0.00: loss=2.466667e+00, accuracy=7.227723e-01
nh=256 & dr=0.20: loss=3.134137e+00, accuracy=6.525653e-01
nh=256 & dr=0.50: loss=2.839252e+00, accuracy=6.894689e-01
nh=512 & dr=0.00: loss=3.212590e+00, accuracy=7.020702e-01
nh=512 & dr=0.20: loss=2.673689e+00, accuracy=7.263726e-01
nh=512